In [5]:
import numpy as np
import pandas as pd
import scanpy as sc
import pyEpoch as Epoch

In [6]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')
adata=sc.read_loom("adMuscle_E12_DPT_071919.loom",sparse=False) #adata matrix cannot be sparse

sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)

mmTFs=pd.read_csv("mmTFs_epoch.csv")
# mmTFs=list(mmTFs["mmTFs"].values)

scanpy==1.9.0 anndata==0.8.0 umap==0.5.2 numpy==1.21.5 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 python-igraph==0.9.9 pynndescent==0.5.6
normalizing by total count per cell
    finished (0:00:00): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


 /Users/wenwen/opt/anaconda3/envs/cahanlab/lib/python3.7/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:Transforming to str index.


In [9]:
def crossweight(adata, lag, minimum, maximum, symmetric_filter=False, weightThresh=0):

    grnDF = adata.uns['grnDF'].copy()

    expDat = makeExpMat(adata)
    #dgenes = adata.uns['dgenes'].copy()
    cells = adata.uns['cells'].index.tolist()

    expDat = expDat[cells]

    # genes=adata.var.index
    # expDat=pd.DataFrame(adata.X).T
    # expDat.columns=adata.obs.index
    # expDat.index=genes
    # expDat=expDat.loc[expDat.sum(axis=1)!=0]
    # expSmoothed=expDat

    
    lag=math.floor(len(expDat.columns)/5)
    minimum=math.ceil(len(expDat.columns)/50)
    maximum=math.floor(len(expDat.columns)/12)
    offset=grnDF.apply(cross_corr,axis=1,expSmoothed=expDat,lag=lag)
    #print(offset)
    grnDF["offset"]=offset
    weighted_scores=[]
    for i in np.arange(grnDF.shape[0]):
        new=score_offset(grnDF["zscore"][i],grnDF["offset"][i],expDat)
        weighted_scores.append(new)
    grnDF["weighted_score"]=weighted_scores
    grnDF = grnDF[grnDF["weighted_score"]>weightThresh]
    adata.uns["grnDF"]=grnDF
    print("Done. Cross-weighted and updated GRN stored in .uns['grnDF'].")
    return adata

In [10]:
#Find Dynamically Expressed Genes
adata=Epoch.findDynGenes(adata, group_column="leiden",pseudotime_column="dpt_pseudotime")

# Optional: to analyze a subset of the data along a particular path you can specify the path
# adata=Epoch.findDynGenes(adata, group_column="leiden", path=['1','0'], pseudotime_column="dpt_pseudotime")

# Reconstruct and perform optional crossweighting
adata=Epoch.reconstructGRN(adata,mmTFs,zThresh=3)
adata=crossweight(adata)

starting gamma...
Done. Dynamic pvalues stored in .uns['genes']. Ordered cells and pseudotime stored in .uns['cells'].
Dynamic genes stored in .uns['dgenes'].
Done. Static GRN stored in .uns['grnDF']


TypeError: crossweight() missing 3 required positional arguments: 'lag', 'minimum', and 'maximum'